In [ ]:
import opensrane as opr
import matplotlib.pyplot as plt

In [ ]:
opr.wipe()
DTobj=opr.DateAndTime.DateTime(1)

windobj=opr.WindData.WindRose(1)

windobj.WindDayClassList=['F','D','B','E','D','D']  
windobj.WindNightClassList=['F','D','B','E','D','D']  

windobj.AlphaCOEFlist=[0.6,0.25,0.15,0.4,0.25,0.25]

windobj.DayWindSpeedList=[[1,2],[2,3],[3,5],[5,7],[7,9],[9,9]]
windobj.NightWindSpeedList=[[1,2],[2,3],[3,5],[5,7],[7,9],[9,9]]

#You don't need to define calmn Condition and program will understand it automatically
windobj.DayWindFreqMatrix=[ [0.446,0.372,0.355,0.109,0.017,0],
                            [0.44,0.938,1.55,0.755,0.097,0.029],
                            [0.898,1.321,3.06,1.402,0.767,0.892],
                            [0.875,1.241,2.626,1.51,0.892,0.646],
                            [0.801,0.927,1.63,0.658,0.355,0.097],
                            [0.87,1.121,0.984,0.309,0.023,0.029],
                            [0.778,0.801,0.91,0.315,0.029,0],
                            [0.652,0.875,1.35,0.498,0.086,0.023],
                            [0.566,0.887,1.659,0.709,0.149,0],
                            [0.583,0.807,2.128,2.998,1.041,0.137],
                            [0.898,1.093,2.408,2.059,1.327,0.154],
                            [1.985,2.088,2.488,1.098,0.332,0.069],
                            [4.067,3.123,1.442,0.292,0.063,0.011],
                            [3.93,5.372,3.85,1.201,0.349,0.057],
                            [1.71,1.619,2.38,0.767,0.109,0.006],
                            [0.698,0.469,0.383,0.154,0.011,0],
                        ]   


windobj.NightWindFreqMatrix=windobj.DayWindFreqMatrix


# #After Feeding the wind Object Below data is not active and the will be active by calling CalcDirectionProbabilities method
# print(windobj.DayCalmn)
# print(windobj.DayDirectProbability)
# print(windobj.DayTheta)
# print(windobj.NightCalmn)
# print(windobj.NightDirectProbability)
# print(windobj.NightTheta)

# windobj.CalcDirectionProbabilities()   #Calling CalcDirectionProbabilities method and the above properties will be calculated
#                                        #It is not nessecary to run this method along the code and program will calculate it whenever it needs! 

# print(windobj.DayCalmn)
# print(windobj.DayDirectProbability)
# print(windobj.DayTheta)
# print(windobj.NightCalmn)
# print(windobj.NightDirectProbability)
# print(windobj.NightTheta)

# a=windobj.GetRandomWindِSample()
# print(a)

#Sampling DayTime
DTobj.SampleisDay()

#Generate a sample of wind
rslt=windobj.GetRandomWindِSample() 
print('Sample: ',rslt)

opr.Plot.Plotly.PlotWindRose(1)

#####################################
# Drawing Code was Only for Day
# Dont forget plot for night and expand the code
####################################

In [ ]:
opr.Plot.Plotly.PlotWindRose(1,Draw_For_Day=False)

In [ ]:
#Verification For Day---------------------------------------
N=100000
windDir=set()
WindDirData=dict()
WindSpeedFreq=[[0.0,0.0,0.0,0.0,0.0,0] for i in range(16)]

#Sampling DayTime
DTobj.SampleisDay()
WindDirData['isCalmn']=0
for i in range(N):
    
    rslt=windobj.GetRandomWindِSample(); #Calculate the a sample
#     print(rslt)
    
    #Calculate wind Direction----------------------------------------------
    if rslt['isCalmn']==False: windDir.add(rslt['WindDirection'])
    if len(windDir)!=len(WindDirData)-1 and rslt['isCalmn']==False:
        wdir=list(windDir)
        add=[i for i in wdir if i not in WindDirData.keys()][0]
        WindDirData[add]=0
        
    
    if rslt['isCalmn']== True:
        WindDirData['isCalmn'] =WindDirData['isCalmn']+1
    else:
        WindDirData[rslt['WindDirection']] =WindDirData[rslt['WindDirection']]+1
    
        
    #Calculating Wind Speed Frequency
    if rslt['isCalmn']==False:
        
        WindRow=[i for i in range(len(windobj.DayTheta)) if windobj.DayTheta[i]==rslt['WindDirection']][0]
        WindCol=[i for i in range(len(windobj.DayWindSpeedList)) if (windobj.DayWindSpeedList[i][0]<=rslt['WindSpeed']<=windobj.DayWindSpeedList[i][1])][0]
#         print(f'({WindRow},{WindCol})')
        WindSpeedFreq[WindRow][WindCol]=WindSpeedFreq[WindRow][WindCol]+1
        
#Normalize Wind MAtrix
for i in range(len(WindSpeedFreq)):
    for j in range(len(WindSpeedFreq[0])):
        WindSpeedFreq[i][j]=WindSpeedFreq[i][j]/N*100
        
print('Monte Carlo Sampling Results: \n',  WindSpeedFreq)
print('\n Defined Values: \n',windobj.DayWindFreqMatrix)
print()

#Normalizing Wind Direction Data        
for key in WindDirData.keys():
    WindDirData[key]=WindDirData[key]/N*100
    



print('\nComparision of Direction Sampling Ratio and User Defined Values:\n')
userdata=dict(zip(windobj.DayTheta,windobj.DayDirectProbability));
for keys in userdata.keys():
    print(f'for direction {keys} average of \t sampled values: {round(WindDirData[keys],3)} \t and UserDefined value: {round(userdata[keys],3)}')
print(f"and Calm condition is= {WindDirData['isCalmn']}")


print()
